# LINEAR_ASSIGN

## Overview
The LINEAR_ASSIGN function solves the classic assignment problem (Hungarian algorithm) using the [`scipy.optimize.linear_sum_assignment`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html) method. This algorithm finds the optimal assignment that minimizes the total cost for a given cost matrix, which is widely used in operations research, scheduling, and logistics.

The Hungarian algorithm operates on a cost matrix $C$ of size $m \times n$ and seeks a one-to-one assignment between rows and columns that minimizes the total cost. The algorithm works by:
- Subtracting the row minima from each row.
- Subtracting the column minima from each column.
- Covering all zeros in the matrix using a minimum number of lines.
- Adjusting the uncovered elements and repeating the process until an optimal assignment is found.

Mathematically, the goal is to find a permutation $\sigma$ that minimizes the total cost:
```math
\min_{\sigma} \sum_{i=1}^n c_{i,\sigma(i)}
```
where $c_{i,j}$ is the cost of assigning worker $i$ to task $j$ and $\sigma$ is a permutation of assignments. The algorithm guarantees an optimal solution in polynomial time and can handle both square and rectangular matrices.

This example function is provided as-is without any representation of accuracy.

## Usage
To use the LINEAR_ASSIGN function in Excel, enter it as a formula in a cell, specifying your cost matrix as a 2D list or Excel range:

```excel
=LINEAR_ASSIGN(cost_matrix)
```
- `cost_matrix` (2D list, required): The cost matrix (m x n) for the assignment. Example: `{4,1,3;2,0,5;3,2,2}`

The function returns a 2D list of [row, col] pairs representing the optimal assignment, or an error message string if calculation fails.

## Examples

### Assigning Workers to Tasks
Assign 3 workers to 3 tasks to minimize total cost.

```excel
=LINEAR_ASSIGN({4,1,3;2,0,5;3,2,2})
```
Expected output:

| Row | Col |
|-----|-----|
|  0  |  1  |
|  1  |  0  |
|  2  |  2  |

### Scheduling Jobs with Unequal Tasks
Assign 3 workers to 4 jobs to minimize total cost (rectangular matrix).

```excel
=LINEAR_ASSIGN({10,19,8,15;10,18,7,17;13,16,9,14})
```
Expected output:

| Row | Col |
|-----|-----|
|  0  |  2  |
|  1  |  0  |
|  2  |  1  |

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def linear_assign(cost_matrix):
    """
    Solves the linear assignment problem (Hungarian algorithm) for a given cost matrix.

    Args:
        cost_matrix (list[list[float]]): 2D list representing the cost matrix.

    Returns:
        list[list[int]]: 2D list of [row, col] assignments, or error message string.

    This example function is provided as-is without any representation of accuracy.
    """
    try:
        arr = np.array(cost_matrix)
        if arr.ndim != 2:
            return "Input must be a 2D list (matrix)."
        row_ind, col_ind = linear_sum_assignment(arr)
        assignment = [[int(r), int(c)] for r, c in zip(row_ind, col_ind)]
        return assignment
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    [[[4, 1, 3], [2, 0, 5], [3, 2, 2]]],
    [[[10, 19, 8, 15], [10, 18, 7, 17], [13, 16, 9, 14]]]
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str, int)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str, int)) for x in row) for row in val)
    return False

import pytest
@pytest.mark.parametrize("cost_matrix", demo_cases)
def test_demo_cases(cost_matrix):
    result = linear_assign(cost_matrix)
    print(f"test_demo_cases output: {result}")
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_input_not_2d():
    cost_matrix = [1, 2, 3]
    result = linear_assign(cost_matrix)
    print(f"test_invalid_input_not_2d output: {result}")
    assert isinstance(result, str) and len(result) > 0

def test_empty_matrix():
    cost_matrix = []
    result = linear_assign(cost_matrix)
    print(f"test_empty_matrix output: {result}")
    assert isinstance(result, str) and len(result) > 0

ipytest.run('-s')

In [ ]:
# Interactive Demo
import gradio as gr

headers = ["col0", "col1", "col2", "col3"]

demo = gr.Interface(
    fn=linear_assign,
    inputs=gr.Dataframe(
        headers=headers,
        label="Cost Matrix",
        row_count=3,
        col_count=4,
        type="array",
        value=demo_cases[1][0]
    ),
    outputs=gr.Dataframe(headers=["Row", "Col"], label="Assignment"),
    examples=demo_cases,
    description="Solve the linear assignment problem (Hungarian algorithm) for a given cost matrix. This demo is provided as-is without any representation of accuracy.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()